In [ ]:
from Explanations_Models.Custom_DT.LIME import LIME
from FeatureImportance.FI import FeatureImportance
import yaml
import pandas as pd
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
import json
import warnings
from tqdm import tqdm
import torch
# Suppress all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

yaml.add_constructor("!join", join)
parser = argparse.ArgumentParser()

parser.add_argument("--config_filename", required=False, type=str)
args = parser.parse_args()
domain = args.config_filename
normalize = True

config_file = open("config_envs/{}.yml".format(domain))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format(domain), encoding="utf8"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights(PATH = None)


config["FI"]["method"] = "LRP"
config["sampler"]["num_samples"] = 1000
config["normalize_FI"] = normalize
FI_calc = FeatureImportance(config, Runner.model.network)
DTMode = LIME(config, Runner, FI_getta=FI_calc)
X,Y = DTMode.sample_set()
out,FI_in_LRP = DTMode.surr_model.model.get_FI(X)

config["FI"]["method"] = "FD"
config["sampler"]["num_samples"] = 1000
config["normalize_FI"] = normalize
FI_calc = FeatureImportance(config, Runner.model.network)
DTMode = LIME(config, Runner, FI_getta=FI_calc)
out,FI_in_FD = DTMode.surr_model.model.get_FI(X)



X = pd.DataFrame(X, columns=config["picture"]["labels"])
if config["surrogate"]["classifier"]:
    Y = pd.DataFrame(Y, columns=["out"])
else:
    Y = pd.DataFrame(Y, columns=config["picture"]["class_names"])



dfs = [X, Y, FI_in_LRP, FI_in_FD, out]

# Folder path
folder_path = "output_folder/" + domain +"norm"+str(config["normalize_FI"])+ "/"

# Check if the folder exists, if not, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save each DataFrame in the folder
names = ["X", "Y", "FI_LRP", "FI_FD", "out"]
for i, df in enumerate(dfs, 1):
    print(i)
    df.to_csv(f"{folder_path}/"+names[i-1]+".csv", index=False)

print("DataFrames saved successfully!")


device:  cuda


c:\Users\Bryan Lavender\scripts\BryanClaim\DTStuff3\InterpretableModelExploration\EnvRunner.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.baseline_network.

1
2
3
4
5
DataFrames saved successfully!
device:  cuda


c:\Users\Bryan Lavender\scripts\BryanClaim\DTStuff3\InterpretableModelExploration\EnvRunner.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.baseline_network.

In [ ]:
normalize = False
for domain in ["cartpole", "acrobot", "_bipedal", "_lunar_lander"]:
    config_file = open("config_envs/{}.yml".format(domain))
    config = yaml.load(config_file, Loader=yaml.FullLoader)
    config.update(yaml.load(open("config_explanations/{}.yml".format(domain), encoding="utf8"), Loader= yaml.FullLoader))
    Runner = GymRunner(config)
    Runner.load_weights(PATH = None)


    config["FI"]["method"] = "LRP"
    config["sampler"]["num_samples"] = 1000
    config["normalize_FI"] = normalize
    FI_calc = FeatureImportance(config, Runner.model.network)
    DTMode = LIME(config, Runner, FI_getta=FI_calc)
    X,Y = DTMode.sample_set()
    out,FI_in_LRP = DTMode.surr_model.model.get_FI(X)

    config["FI"]["method"] = "FD"
    config["sampler"]["num_samples"] = 1000
    config["normalize_FI"] = normalize
    FI_calc = FeatureImportance(config, Runner.model.network)
    DTMode = LIME(config, Runner, FI_getta=FI_calc)
    out,FI_in_FD = DTMode.surr_model.model.get_FI(X)



    X = pd.DataFrame(X, columns=config["picture"]["labels"])
    if config["surrogate"]["classifier"]:
        Y = pd.DataFrame(Y, columns=["out"])
    else:
        Y = pd.DataFrame(Y, columns=config["picture"]["class_names"])



    dfs = [X, Y, FI_in_LRP, FI_in_FD, out]

    # Folder path
    folder_path = "output_folder/" + domain +"norm"+str(config["normalize_FI"])+ "/"

    # Check if the folder exists, if not, create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Save each DataFrame in the folder
    names = ["X", "Y", "FI_LRP", "FI_FD", "out"]
    for i, df in enumerate(dfs, 1):
        print(i)
        df.to_csv(f"{folder_path}/"+names[i-1]+".csv", index=False)

    print("DataFrames saved successfully!")


In [1]:
import pandas as pd
import os

# Folder path where CSV files are stored


def get_dataframes(domain, max_number):
    normalize = True
    folder_path = "output_folder_tmp/" + domain +"norm"+str(normalize)+ "/"
    # File prefixes for each set of files
    def read_multi_headed_csv(file_path):
        return pd.read_csv(file_path, header=[0, 1])  # This assumes a 2-level header

    # Concatenate files for multi-index columns like FI_FD and FI_LRP
    file_types = ['X', 'Y', 'FI_FD', 'FI_LRP', 'out']

    dataframes = {}
    for file_type in file_types:
        file_list = [f"{folder_path}/{file_type}{i}.csv" for i in range(1, max_number+1)]  # Adjust range as needed

        # Use read_multi_headed_csv for FI_FD and FI_LRP
        if file_type in ['FI_FD', 'FI_LRP']:
            df_list = [read_multi_headed_csv(file) for file in file_list]
        else:
            df_list = [pd.read_csv(file) for file in file_list]

        # Concatenate them, preserving the multi-index headers
        dataframes[file_type] = pd.concat(df_list, ignore_index=True)



    #final_dataframe = pd.concat([dataframes['X'], dataframes['Y'], dataframes['FI_FD'], dataframes['FI_LRP'], dataframes['out']], axis=1)


    #final_dataframe.to_csv(f"{folder_path}/final_concatenated_data.csv", index=False)


    return dataframes



In [ ]:
import random

domain = "_bipedal"
max_number = 20
dataframes = get_dataframes(domain, max_number)
for i in range(0,100):
    unique_indices = random.sample(range(0, 1001), 100)
    
    X = dataframes["X"].loc[unique_indices]
    Y = dataframes["Y"].loc[unique_indices]
    FI_FD = dataframes["FI_FD"].loc[unique_indices]
    FI_LRP = dataframes["FI_LRP"].loc[unique_indices]

    config["surrogate"]["use_FI"] = False
    config["sampler"]["num_samples"] = 100
    config["FI"]["grouping"] = "Baseline"
    DTMode = LIME(config, Runner)
    config["surrogate"]["multi_tree"] = False
    DTMode.surr_model.fit(X,Y, FI_in=FI_in, out_logits=out)
    # DTMode.percent_Correct()
    # DTMode.uniform_Correct()
    mets = DTMode.get_metrics()
    
    with open(fileLoc + config["FI"]["grouping"] + str(samps), "a") as file:
        jstring = json.dumps(mets)
        file.write(jstring + '\n')
    DTMode.surr_model.Save(FilenameEnder="tree" + fileLoc+config["FI"]["grouping"] + str(samps)+str(ep)+".json")